<a href="https://colab.research.google.com/github/gmelaku/GM/blob/master/PS12_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment Analysis of 2020 US Presidential Candidate's twitter feed

#### By Gebremedhin Melaku

Install Tweepy and textblob and import the necessary libraries

In [0]:
!pip install tweepy
!pip install textblob
import tweepy
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re


##### Authenticate your tweepy API

In [0]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)
CONSUMER_KEY = "WlPt2p3U5gXb9jAeipEhaec7p"
CONSUMER_SECRET = "Z5ij8szjP399I1RrY6DJIGQ6nK63u7XJioI1osgECc3InsSQ6G"
ACCESS_TOKEN = "762379377858453504-Psv8LQRbN3ID2IQYCq6Pn29xPnDOCjI"
ACCESS_TOKEN_SECRET = "7mXxiVv7Ohx6vVgFmr77pBNODDoJW5LgDQ9GhrE2IE2FA"

##### Create a Twitter Client

We will create a Twitter client that will define the different APIs for analysis

In [0]:
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


#### Twitter Streamer

In [0]:
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


#### Twitter Stream Listener

In [0]:
class TwitterListener(StreamListener):
   
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


#### Tweet Analyzer, cleaning and defining the polarity of the sentiment

We will define sentiment analysis using TextBlob() function. We will also define the polarity of the tweet sentiment where >1 is regarded as a positive sentiment and <1 as a negative sentiment. a value of 0 will be taken as the tweet being neutral.

We will also define a clean_tweet by removing all the special characters within each tweets

In [0]:
class TweetAnalyzer():

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df

#### Analyze Sentiment for the Candidates

##### Sentiment Analysis of the last 200 tweets of Donald J Trump

First we will add an additional Column named "sentiment".

In [0]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="realDonaldTrump", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df.head(10))

tweets ... sentiment
0 Such a wonderful reception yesterday in Pennsy... ... 1
1 I am proud to announce that the United States ... ... 1
2 Today I was presented with the brand new @Spac... ... 1
3 RT @WhiteHouse: Recent data "made me feel even... ... 1
4 RT @WhiteHouse: Operation Warp Speed is "a mas... ... 0
5 RT @WhiteHouse: LIVE: POTUS Delivers Remarks o... ... 1
6 Chicago’s @CTA has big federal resources headi... ... 0
7 I am proud to announce that @NJTransit will re... ... 1
8 ....Likewise, the @nytimes and @washingtonpost... ... -1
9 Thank you to @foxandfriends for covering, supr... ... 1

[10 rows x 8 columns]

##### Sentiment visualization of the last 200 tweets of Donald J Trump

In [0]:
import matplotlib.pyplot as plt
time_likes = pd.Series(data=df['likes'].values, index=df['date'])
time_likes.plot(figsize=(16, 4), label="likes", legend=True)
time_sentiment = pd.Series(data=df['sentiment'].values, index=df['date'])
time_sentiment.plot(figsize=(16, 4), label="sentiment", legend=True)
plt.show()

##### Average sentiment for the last 200 tweets of Donald J Trump

In [0]:
print(np.mean(df['sentiment']))

0.24242424242424243

##### Sentiment Analysis of the last 200 tweets of Joe Biden

In [0]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="JoeBiden", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df.head(10))

tweets ... sentiment
0 To everyone who has lost a loved one due to CO... ... 1
1 If you’re looking for ways to show off your su... ... 1
2 We have the power to create the future we want... ... 0
3 To all of our front line workers who are worki... ... 0
4 We are months into this crisis. There is simpl... ... -1
5 Tonight, @StaceyAbrams and I will be joining @... ... 0
6 What he said. https://t.co/qfK5oHyFba ... 0
7 Donald Trump can deflect and shift blame all h... ... -1
8 Sherian, a small business owner from South Car... ... -1
9 The lives of tens of millions of Americans hav... ... 1

[10 rows x 8 columns]

##### Sentiment visualization of the last 200 tweets of Joe Biden

In [0]:
time_likes = pd.Series(data=df['likes'].values, index=df['date'])
time_likes.plot(figsize=(16, 4), label="likes", legend=True)
time_sentiment = pd.Series(data=df['sentiment'].values, index=df['date'])
time_sentiment.plot(figsize=(16, 4), label="sentiment", legend=True)
plt.show()

##### Average sentiment for the last 200 tweets of Joe Biden

In [0]:
print(np.mean(df['sentiment']))

0.27